In [1]:
import os

## GAN Variants
# from GAN import GAN
# from CGAN import CGAN
# from infoGAN import infoGAN
# from ACGAN import ACGAN
# from EBGAN import EBGAN
# from WGAN import WGAN
from WGAN_GP_withWM import WGAN_GP
# from DRAGAN import DRAGAN
# from LSGAN import LSGAN
# from BEGAN import BEGAN

## VAE Variants
# from VAE import VAE
# from CVAE import CVAE

from utils_withWM import show_all_variables
from utils_withWM import check_folder

import tensorflow as tf
import argparse

import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 程序最多只能占用指定gpu90%的显存
#config.gpu_options.allow_growth = True      #程序按需申请内存
sess = tf.Session(config = config)

In [2]:
"""parsing and configuration"""
def parse_args():
    desc = "Tensorflow implementation of GAN collections"
    parser = argparse.ArgumentParser(description=desc)
    
    #args = parser.parse_args(args=[])
    
    parser.add_argument('--gan_type', type=str, default='GAN',
                        choices=['GAN', 'CGAN', 'infoGAN', 'ACGAN', 'EBGAN', 'BEGAN', 'WGAN', 'WGAN_GP', 'DRAGAN', 'LSGAN', 'VAE', 'CVAE'],
                        help='The type of GAN')
                        #help='The type of GAN', required=True)
    parser.add_argument('--dataset', type=str, default='mnist', choices=['mnist', 'fashion-mnist', 'celebA', 'Anime'],
                        help='The name of dataset')
    parser.add_argument('--epoch', type=int, default=200, help='The number of epochs to run')
    parser.add_argument('--batch_size', type=int, default=128, help='The size of batch')
    parser.add_argument('--z_dim', type=int, default=128, help='Dimension of noise vector')
    parser.add_argument('--checkpoint_dir', type=str, default='./ResultWithWM/checkpoint',
                        help='Directory name to save the checkpoints')
    parser.add_argument('--result_dir', type=str, default='./ResultWithWM/results',
                        help='Directory name to save the generated images')
    parser.add_argument('--log_dir', type=str, default='./ResultWithWM/logs',
                        help='Directory name to save training logs')
    parser.add_argument('--mode', type=str, default='train',
                        help='train or infer')
    parser.add_argument('--testing_tags', type=str, default='./AnimeDataset/sample_testing_text.txt',
                        help='train or infer')
    return check_args(parser.parse_args(args=['--gan_type','WGAN_GP','--dataset','Anime','--mode','train',
                                              '--testing_tags','./extra_data/test']))
    #return check_args(parser.parse_args())

"""checking arguments"""
def check_args(args): 
    # --checkpoint_dir
    check_folder(args.checkpoint_dir)

    # --result_dir
    check_folder(args.result_dir)

    # --result_dir
    check_folder(args.log_dir)

    # --epoch
    assert args.epoch >= 1, 'number of epochs must be larger than or equal to one'

    # --batch_size
    assert args.batch_size >= 1, 'batch size must be larger than or equal to one'

    # --z_dim
    assert args.z_dim >= 1, 'dimension of noise vector must be larger than or equal to one'

    return args

In [3]:
"""main"""
def main():
    # Fix random seed.
    tf.set_random_seed(9487)
    #tf.random.set_seed(9487)
    
    # parse arguments
    args = parse_args()
    if args is None:
          exit()

    # open session
    # models = [GAN, CGAN, infoGAN, ACGAN, EBGAN, WGAN, WGAN_GP, DRAGAN,
    #           LSGAN, BEGAN, VAE, CVAE]
    models = [WGAN_GP] 
    #with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True)) as sess:    
        # declare instance for GAN

        gan = None
        for model in models: # 查找该模型属于哪一个GAN
            if args.gan_type == model.model_name:
                gan = model(sess,
                            epoch=args.epoch,
                            batch_size=args.batch_size,
                            z_dim=args.z_dim,
                            dataset_name=args.dataset,
                            checkpoint_dir=args.checkpoint_dir,
                            result_dir=args.result_dir,
                            log_dir=args.log_dir, 
                            mode=args.mode)
        if gan is None:
            raise Exception("[!] There is no option for " + args.gan_type)

        # build graph
        gan.build_model()

        # show network architecture
        show_all_variables()
        if args.mode == 'train':
            # launch the graph in a session
            gan.train()
            print(" [*] Training finished!")

            # # visualize learned generator
            # gan.visualize_results(args.epoch-1)
            # print(" [*] Testing finished!")
        elif args.mode == 'infer': # 我找到测试的代码了
            tag_dict = ['orange hair', 'white hair', 'aqua hair', 'gray hair', 'green hair', 'red hair', 'purple hair', 
                        'pink hair', 'blue hair', 'black hair', 'brown hair', 'blonde hair', 'no hair',
                        'gray eyes', 'black eyes', 'orange eyes', 'pink eyes', 'yellow eyes',
                        'aqua eyes', 'purple eyes', 'green eyes', 'brown eyes', 'red eyes', 'blue eyes','no eyes']
            testing_tags_txt = open(args.testing_tags, 'r').readlines()

            test_labels = np.zeros((args.batch_size, len(tag_dict)))
            for line in testing_tags_txt:
                id, tags = line.split(',')
                label = np.zeros(len(tag_dict))
                
                for i in range(len(tag_dict)):
                    if tag_dict[i] in tags:
                        label[i] = 1
                test_labels[int(id) - 1] = label
                
            # visualize learned generator
            gan.infer(test_labels)
            print(" [*] Infer finished!")

print("finish")

finish


In [4]:
if __name__ == '__main__':
    main()


[Tip 14] Train discriminator more (sometimes)
images.shape:  (36940, 64, 64, 3)
labels.shape:  (36940, 25)
Loading images to numpy array...
Random shuffling images and labels...
[Tip 1] Normalize the images between -1 and 1.


W0410 12:52:00.041800 140670769506112 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:160: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0410 12:52:00.043135 140670769506112 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:168: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0410 12:52:00.103230 140670769506112 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:173: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0410 12:52:00.149402 140670769506112 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:132: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



[Tip Hung-yi Lee] Data Augmentation (flip images horizontally + rotate images)
[Tip 4] BatchNorm
[Tip 5] Avoid Sparse Gradients: ReLU, MaxPool


W0410 12:52:00.466180 140670769506112 deprecation.py:323] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:141: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0410 12:52:01.477191 140670769506112 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:198: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.



[Tip 5] Avoid Sparse Gradients: ReLU, MaxPool


...

Epoch: [ 198/ 200] [ 287/ 288] time: 18492.9678, d_loss: 0.25932896, g_loss: 1.62349224


W0410 18:00:18.758986 140670769506112 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Epoch: [ 199/ 200] [ 287/ 288] time: 18578.6455, d_loss: 0.16899064, g_loss: 2.70188761


W0410 18:01:44.307610 140670769506112 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


 [*] Training finished!


In [5]:
# '--testing_tags', type=str, default='./AnimeDataset/sample_testing_text.txt'
import gc
gc.collect()

0